In [1]:
import os
import time
import re
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException



In [135]:

def scroll_shim(passed_in_driver, object):
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % ( x, y)
    scroll_nav_out_of_way = 'window.scrollBy(0, -73);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)
    

def asignar_variables_carrefour(features):
    
    title = price = main_image = marca = tipo = dimensiones = float('nan')
        
    i = 0 
    
    for feature in features:
        i = i + 1
        
        
        if feature == 'Características':
            caracteristicas = features[i]
        elif feature == 'Tipo':
            tipo = features[i]
        elif feature == 'Dimensiones del producto (AltoxAnchoXFondo)':
            dimensiones = features[i]
        
    
    return caracteristicas , tipo , dimensiones


def asignar_variables_ebay(detalles_tecnicos_1, detalles_tecnicos_2, detalles_tecnicos_3, detalles_tecnicos_4, detalles_tecnicos_5, detalles_tecnicos_6, detalles_tecnicos_7):
    
    title = price = main_image = estado = marca = tipo = caracteristicas = float('nan')
        
    
    for detalle_tecnico in detalles_tecnicos_1:
        if detalle_tecnico[0] == 'Estado:':
            estado = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            tipo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            caracteristicas = detalle_tecnico[1]
      
    
    for detalle_tecnico in detalles_tecnicos_2:
        if detalle_tecnico[0] == 'Estado:':
            estado = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            tipo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            caracteristicas = detalle_tecnico[1]
      
    for detalle_tecnico in detalles_tecnicos_3:
        if detalle_tecnico[0] == 'Estado:':
            estado = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            tipo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            caracteristicas = detalle_tecnico[1]
      
            
    for detalle_tecnico in detalles_tecnicos_4:
        if detalle_tecnico[0] == 'Estado:':
            estado = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            tipo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            caracteristicas = detalle_tecnico[1]
      
            
    for detalle_tecnico in detalles_tecnicos_5:
        if detalle_tecnico[0] == 'Estado:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            numero_de_modelo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            valoracion = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            clasificacion = detalle_tecnico[1]
            
    for detalle_tecnico in detalles_tecnicos_6:
        if detalle_tecnico[0] == 'Estado:':
            estado = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            tipo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            caracteristicas = detalle_tecnico[1]
      
            
    for detalle_tecnico in detalles_tecnicos_7:
        if detalle_tecnico[0] == 'Estado:':
            estado = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Marca:':
            marca = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'Tipo:':
            tipo = detalle_tecnico[1]
        elif detalle_tecnico[0] == 'CaracterÃ\xadsticas:':
            caracteristicas = detalle_tecnico[1]
      
    
    return estado, marca, tipo, caracteristicas


def initialize_ebay_info_df():
    
    ebay_info = pd.DataFrame(columns = ['item_name.value', 
                                                          'list_price.value_with_tax', 
                                                          'main_image', 'estado', 'marca',
                                                          'tipo','caracteristicas'
                                                         ])
    return ebay_info


def initialize_carrefour_info_df():
    
    carrefour_info = pd.DataFrame(columns = ['item_name.value', 
                                                          'list_price.value_with_tax', 
                                                          'main_image', 'caracteristicas',
                                                          'tipo', 'dimensiones'
                                                         ])
    return carrefour_info



In [136]:
os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'

ean_list= ['8436028922536', '8436551456690', '3397701245487', '8435178361905']

In [143]:
def carrefour(ean_list):
    
    
    carrefour_info = initialize_carrefour_info_df()


    for ean in ean_list:
        
        title = price = main_image = caracteristicas = tipo = dimensiones = float('nan')

        
        browser = webdriver.Firefox()
        browser.get('https://www.carrefour.es/')
        
        time.sleep(3)

        action = ActionChains(browser)
        click_browser  = browser.find_element_by_xpath('//input[@id="atg_store_searchInput"]')
        action.move_to_element((click_browser)).click().perform()

        time.sleep(3)

        search = browser.find_element_by_xpath('//input[@class="ebx-search-box__input ebx-search-box__input-query"]')
        search.clear()
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)
        
        try: 

            action = ActionChains(browser)
            click_image  = browser.find_element_by_xpath("(//img[@class='ebx-result-figure__img'])")
            action.move_to_element((click_image)).click().perform()

        except NoSuchElementException:
            
            print(f"Item {ean} not found")
            
        
        time.sleep(3)
        
        try: 

            title = browser.find_elements_by_xpath('//h1[@class="product-header__name"]')[0].text
        
        except NoSuchElementException:
            
            print(f"title {ean} not found")
            
        except IndexError:
            
            print(f"title {ean} not found")
            
            
        
        try: 
            
            price = browser.find_elements_by_xpath('//span[@class="buybox__price--current"]')[0].text
        
        except NoSuchElementException:
            
            price = browser.find_elements_by_xpath('//span[@class="buybox__price"]')[0].text

        except NoSuchElementException:
            
            print(f"price {ean} not found")
        
        except IndexError:
            
            print(f"price {ean} not found")
            
        
        try: 
            
            main_image = browser.find_element_by_xpath('//img[@class="main-image__image"]').get_attribute('src')

        
        except NoSuchElementException:
            
            print(f"main image {ean} not found")
        
        except IndexError:
            
            print(f"main image {ean} not found")
            
        
        try: 
            
            action = ActionChains(browser)
            click_view_more  = browser.find_element_by_xpath("(//p[@class='product-details__show'])")
            if 'firefox' in browser.capabilities['browserName']:
                scroll_shim(browser, click_view_more)
            action.move_to_element((click_view_more)).click().perform()
            
            time.sleep(1)
            
            features = browser.find_elements_by_xpath('//div[@class="product-details"]')[0].text
            features_list=features.split("\n")

            caracteristicas, tipo, dimensiones = asignar_variables_carrefour(features_list)
            
        except NoSuchElementException:
            
            print(f"features image {ean} not found")
        
        except IndexError:
            
            print(f"features image {ean} not found")
        

        
        carrefour_info.loc[ean] = [title, price, main_image, caracteristicas, tipo , dimensiones]

            
        
    browser.close()
    
    return carrefour_info
                


In [144]:
df= carrefour(ean_list)

price 3397701245487 not found


In [145]:
df

,item_name.value,list_price.value_with_tax,main_image,caracteristicas,tipo,dimensiones
8436028922536,Frigorífico Combi Icecool A+ CBNF1854,299 €,https://static.carrefour.es/hd_510x_/crs/cdn_s...,Clasificación energética:A+ Color:Blanco Capac...,Frigorífico,185.50 x 59.50 x 63.20
8436551456690,"Television 32"" Blualta Bl-f32-hd","121,15 €",https://static.carrefour.es/hd_510x_/imagenes/...,- Pantalla LED 32” - Resolución: 1366 x 768 - ...,Televisores,NaN
3397701245487,Mueble Mesa Tv Sagel 2 Cajones 2 Puertas Corre...,NaN,https://static.carrefour.es/hd_510x_/imagenes/...,Mesa de TV de 2 puertas correderas y 2 cajones...,Muebles TV,NaN
8435178361905,"Colchon Visco-elastico Ergo-therapy 90x190,18 ...","94,99 €",https://static.carrefour.es/hd_510x_/imagenes/...,Tapa superior: Realizada con tejido Strech de ...,NaN,NaN
